### Load model
- Catboost
loaded_cat = CatBoostRegressor()
loaded_cat.load_model("catboost_model.cbm")

- XGBoost
loaded_xgbr = XGBRegressor()
loaded_xgbr.load_model("xgboost_model.json")

- LightGBM
import lightgbm as lgb
loaded_lgbmr = lgb.Booster(model_file="lightgbm_model.txt")



In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import regex as re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
#import pygwalker as pyg

train = pd.read_csv("combined.csv")
train.head()

,brand,price,old,new,cpu,cpu_brand,ram_capacity,ram_brand,hard_drive_type,hard_drive_capacity,card,card_brand,screen_size,screen_type
0,Dell,15990000.0,0,1,Intel Core i5 1340P,Intel,16GB,LPDDR5,SSD,512 GB,Intel Iris Xe Graphics,Intel,16,FHD
1,Lenovo,27990000.0,0,1,Intel Core i7 1365U,Intel,16GB,LPDDR5,SSD,256 GB,Intel Iris Xe Graphics,Intel,14,FHD
2,Dell,15990000.0,0,1,Intel Core i5 1340P,Intel,16GB,LPDDR5,SSD,512 GB,Intel Iris Xe Graphics,Intel,14,FHD
3,Lenovo,17990000.0,0,1,AMD Ryzen 7 7735H,AMD,16GB,LPDDR5,SSD,512 GB,AMD Radeon 680M,AMD,14,3K
4,Microsoft,26990000.0,0,1,Intel Core i7 1255U,Intel,16GB,LPDDR5,SSD,256 GB,Intel Iris Xe Graphics,Intel,13,3K


In [256]:
train['brand'] = train['brand'].replace('Apple', 'Macbook')
train['brand'] = train['brand'].replace('MacBook', 'Macbook')
train['brand'] = train['brand'].replace('Thinkpad', 'Lenovo')
train['brand'] = train['brand'].replace('Legion', 'Lenovo')
train['brand'] = train['brand'].replace('Ideapad', 'Lenovo')
train['brand'] = train['brand'].replace('ThinkPad', 'Lenovo')
train['brand'] = train['brand'].replace('LENOVO', 'Lenovo')
train['brand'] = train['brand'].replace('ACER', 'Acer')
train['brand'] = train['brand'].replace('ASUS', 'Asus')


In [257]:
unique_brands = train['brand'].unique()
print(unique_brands)

['Dell' 'Lenovo' 'Microsoft' 'Macbook' 'Acer' 'LG' 'Asus' 'MSI' 'HP'
 'Gaming' nan]


In [258]:
brand_counts = train['brand'].value_counts()
print(brand_counts)

brand
Lenovo       328
Asus         170
Dell         140
HP            84
Acer          79
MSI           72
LG            19
Gaming        18
Macbook       17
Microsoft     10
Name: count, dtype: int64


In [259]:
train.columns

Index(['brand', 'price', 'old', 'new', 'cpu', 'cpu_brand', 'ram_capacity',
       'ram_brand', 'hard_drive_type', 'hard_drive_capacity', 'card',
       'card_brand', 'screen_size', 'screen_type'],
      dtype='object')

In [260]:
cat_cols = ['brand', 'cpu', 'cpu_brand', 'ram_capacity',
       'ram_brand', 'hard_drive_type', 'hard_drive_capacity', 'card',
       'card_brand', 'screen_size', 'screen_type']

In [261]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                937 non-null    object 
 1   price                920 non-null    float64
 2   old                  940 non-null    int64  
 3   new                  940 non-null    int64  
 4   cpu                  937 non-null    object 
 5   cpu_brand            937 non-null    object 
 6   ram_capacity         937 non-null    object 
 7   ram_brand            900 non-null    object 
 8   hard_drive_type      936 non-null    object 
 9   hard_drive_capacity  887 non-null    object 
 10  card                 927 non-null    object 
 11  card_brand           927 non-null    object 
 12  screen_size          934 non-null    object 
 13  screen_type          863 non-null    object 
dtypes: float64(1), int64(2), object(11)
memory usage: 102.9+ KB


In [262]:
print('trainframe encoded by OHE dimension : ', pd.get_dummies(train, columns = cat_cols, drop_first = True).shape)

trainframe encoded by OHE dimension :  (940, 758)


In [263]:
train[cat_cols] = train[cat_cols].fillna('missing')

In [264]:
en = LabelEncoder()

for cols in cat_cols:
    train[cols] = en.fit_transform(train[cols])

print('Dataframe encoded by Label encoding dimension : ', train.shape)

Dataframe encoded by Label encoding dimension :  (940, 14)


In [265]:
train.head()

,brand,price,old,new,cpu,cpu_brand,ram_capacity,ram_brand,hard_drive_type,hard_drive_capacity,card,card_brand,screen_size,screen_type
0,2,15990000.0,0,1,232,5,4,21,6,14,96,14,20,8
1,6,27990000.0,0,1,266,5,4,21,6,9,96,14,9,8
2,2,15990000.0,0,1,232,5,4,21,6,14,96,14,9,8
3,6,17990000.0,0,1,19,0,4,21,6,14,8,5,9,6
4,9,26990000.0,0,1,254,5,4,21,6,9,96,14,3,6


In [266]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                940 non-null    int64  
 1   price                920 non-null    float64
 2   old                  940 non-null    int64  
 3   new                  940 non-null    int64  
 4   cpu                  940 non-null    int64  
 5   cpu_brand            940 non-null    int64  
 6   ram_capacity         940 non-null    int64  
 7   ram_brand            940 non-null    int64  
 8   hard_drive_type      940 non-null    int64  
 9   hard_drive_capacity  940 non-null    int64  
 10  card                 940 non-null    int64  
 11  card_brand           940 non-null    int64  
 12  screen_size          940 non-null    int64  
 13  screen_type          940 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 102.9 KB


In [267]:
print(train['price'].describe())

count    9.200000e+02
mean     2.798950e+07
std      1.804666e+07
min      6.990000e+06
25%      1.699000e+07
50%      2.289000e+07
75%      3.299000e+07
max      1.489900e+08
Name: price, dtype: float64


In [268]:
print("Max value in y_train:", train.max())
print("Min value in y_train:", train.min())


Max value in y_train: brand                         10.0
price                  148990000.0
old                            1.0
new                            1.0
cpu                          338.0
cpu_brand                     16.0
ram_capacity                  16.0
ram_brand                     28.0
hard_drive_type                7.0
hard_drive_capacity           23.0
card                         248.0
card_brand                    28.0
screen_size                   43.0
screen_type                    9.0
dtype: float64
Min value in y_train: brand                        0.0
price                  6990000.0
old                          0.0
new                          0.0
cpu                          0.0
cpu_brand                    0.0
ram_capacity                 0.0
ram_brand                    0.0
hard_drive_type              0.0
hard_drive_capacity          0.0
card                         0.0
card_brand                   0.0
screen_size                  0.0
screen_type           

In [269]:
def clean_price_data(df):
    # Remove any potential negative prices
    df = df[df['price'] > 0]
    
    # Remove extreme outliers using IQR method
    Q1 = df['price'].quantile(0.25)
    Q3 = df['price'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    
    df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
    
    return df

In [270]:
train = clean_price_data(train)

In [271]:
X = train.drop('price', axis = 1)
y = train['price']

In [272]:
print(X.shape)
print(y.shape)

(897, 13)
(897,)


In [273]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.05,random_state=0)

In [274]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(852, 13)
(852,)
(45, 13)
(45,)


In [281]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

In [ ]:
lgbmr = LGBMRegressor()
xgbr = XGBRegressor(
    n_estimators=100,     # Number of trees
    learning_rate=0.1,    # Step size shrinkage
    max_depth=6,          # Maximum tree depth
    random_state=42,      # Ensures reproducibility
    verbosity=1           # Logs progress
)



In [277]:
if np.any(np.isnan(y_train)):
    print("y_train contains NaN values.")
if np.any(np.isinf(y_train)):
    print("y_train contains infinity values.")


In [278]:
lgbmr.fit(X_train,y_train)
xgbr.fit(X_train,y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 462
[LightGBM] [Info] Number of data points in the train set: 852, number of used features: 12
[LightGBM] [Info] Start training from score 25813038.732394
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [282]:
catboost_model = CatBoostRegressor(cat_features=cat_cols, verbose=0)

In [288]:
param_grid = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7]
}

# Perform grid search
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, cv=3, scoring='r2', verbose=1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'depth': 6, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.1}


In [283]:
catboost_model.fit(X_train, y_train)

In [289]:
best_catboost_model = CatBoostRegressor(cat_features=cat_cols, **best_params, verbose=0)
best_catboost_model.fit(X_train, y_train)

In [279]:
xgbr_pred = xgbr.predict(X_test)
print("r2_score",r2_score(y_test,xgbr_pred))

r2_score 0.6448640250111966


In [280]:
lgbmr_pred = lgbmr.predict(X_test)
print("r2_score",r2_score(y_test,lgbmr_pred))

r2_score 0.7639991703043368


In [284]:
y_pred = catboost_model.predict(X_test)
print(f"R2 Score: {catboost_model.score(X_test, y_test)}")

R2 Score: 0.7279588402868794


In [290]:
y_pred = best_catboost_model.predict(X_test)
print(f"R2 Score: {best_catboost_model.score(X_test, y_test)}")

R2 Score: 0.7553168934010216


In [292]:
catboost_model.save_model("model/catboost_model.cbm")

In [294]:
lgbmr.booster_.save_model("model/lightgbm_model.txt")
xgbr.save_model("model/xgboost_model.json")

In [296]:
loaded_cat = CatBoostRegressor()
loaded_cat.load_model("model/catboost_model.cbm")

loaded_xgbr = XGBRegressor()
loaded_xgbr.load_model("model/xgboost_model.json")

import lightgbm as lgb
loaded_lgbmr = lgb.Booster(model_file="model/lightgbm_model.txt")